In [1]:
# Dependencies and set up
import pandas as pd
import numpy as np
import json

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify
from sqlalchemy import inspect

from sqlite3 import connect

# City of Chicago -- Crimes - from 2001 to 2021

In [2]:
# Raw crime table
table = pd.read_csv('Crimes_-_2001_to_Present.csv') 
table.head(2)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,-87.6700,"(41.815117282, -87.669999562)",29.0,14924.0,59.0,706.0,3.0,37.0,23.0,108.0
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,-87.7654,"(41.895080471, -87.765400451)",4.0,4299.0,26.0,562.0,45.0,5.0,25.0,67.0


In [3]:
# table.drop((table['Year'] < 2010).index, inplace=True)

In [4]:
df = table[(table.Year>2010) & (table.Year<2021) & (table['Latitude'].notna())]
df

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,-87.670000,"(41.815117282, -87.669999562)",29.0,14924.0,59.0,706.0,3.0,37.0,23.0,108.0
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,-87.765400,"(41.895080471, -87.765400451)",4.0,4299.0,26.0,562.0,45.0,5.0,25.0,67.0
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,-87.716650,"(41.937405765, -87.716649687)",15.0,21538.0,22.0,216.0,12.0,39.0,7.0,168.0
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,-87.755121,"(41.881903443, -87.755121152)",11.0,22216.0,26.0,696.0,23.0,32.0,25.0,81.0
5,10224742,HY411435,09/05/2015 10:55:00 AM,082XX S LOOMIS BLVD,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,...,-87.658431,"(41.744378879, -87.658430635)",18.0,21554.0,70.0,575.0,13.0,59.0,20.0,237.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7371597,11440856,JB428147,09/09/2018 02:32:00 AM,036XX S ARCHER AVE,041A,BATTERY,AGGRAVATED - HANDGUN,STREET,False,False,...,-87.682498,"(41.827417262, -87.682497902)",26.0,14924.0,56.0,719.0,1.0,37.0,23.0,165.0
7371598,11557153,JC105373,01/05/2019 07:13:00 AM,040XX N ELSTON AVE,0610,BURGLARY,FORCIBLE ENTRY,RESTAURANT,False,False,...,-87.719885,"(41.954198496, -87.719884773)",2.0,21538.0,16.0,364.0,50.0,39.0,1.0,9.0
7371599,12178678,JD382233,09/28/2020 07:00:00 AM,025XX N HARDING AVE,0610,BURGLARY,FORCIBLE ENTRY,APARTMENT,True,False,...,-87.725618,"(41.926885578, -87.725618252)",27.0,22535.0,23.0,186.0,17.0,1.0,6.0,181.0
7371600,12215915,JD425195,11/08/2020 04:00:00 AM,025XX N HAMLIN AVE,0610,BURGLARY,FORCIBLE ENTRY,APARTMENT,True,False,...,-87.721959,"(41.92716253, -87.721959204)",15.0,22535.0,23.0,186.0,17.0,1.0,6.0,181.0


In [5]:
df.reset_index(drop=True, inplace=True)

df

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,-87.670000,"(41.815117282, -87.669999562)",29.0,14924.0,59.0,706.0,3.0,37.0,23.0,108.0
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,-87.765400,"(41.895080471, -87.765400451)",4.0,4299.0,26.0,562.0,45.0,5.0,25.0,67.0
2,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,-87.716650,"(41.937405765, -87.716649687)",15.0,21538.0,22.0,216.0,12.0,39.0,7.0,168.0
3,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,-87.755121,"(41.881903443, -87.755121152)",11.0,22216.0,26.0,696.0,23.0,32.0,25.0,81.0
4,10224742,HY411435,09/05/2015 10:55:00 AM,082XX S LOOMIS BLVD,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,...,-87.658431,"(41.744378879, -87.658430635)",18.0,21554.0,70.0,575.0,13.0,59.0,20.0,237.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2787987,11440856,JB428147,09/09/2018 02:32:00 AM,036XX S ARCHER AVE,041A,BATTERY,AGGRAVATED - HANDGUN,STREET,False,False,...,-87.682498,"(41.827417262, -87.682497902)",26.0,14924.0,56.0,719.0,1.0,37.0,23.0,165.0
2787988,11557153,JC105373,01/05/2019 07:13:00 AM,040XX N ELSTON AVE,0610,BURGLARY,FORCIBLE ENTRY,RESTAURANT,False,False,...,-87.719885,"(41.954198496, -87.719884773)",2.0,21538.0,16.0,364.0,50.0,39.0,1.0,9.0
2787989,12178678,JD382233,09/28/2020 07:00:00 AM,025XX N HARDING AVE,0610,BURGLARY,FORCIBLE ENTRY,APARTMENT,True,False,...,-87.725618,"(41.926885578, -87.725618252)",27.0,22535.0,23.0,186.0,17.0,1.0,6.0,181.0
2787990,12215915,JD425195,11/08/2020 04:00:00 AM,025XX N HAMLIN AVE,0610,BURGLARY,FORCIBLE ENTRY,APARTMENT,True,False,...,-87.721959,"(41.92716253, -87.721959204)",15.0,22535.0,23.0,186.0,17.0,1.0,6.0,181.0


In [6]:
# df = df.drop(['index'], axis=1)
# df

In [7]:
# Groupby Primary type
# df = table.groupby(['Primary Type']).count()['Description']
# df

In [8]:
# Dropping columns
table_db = df[['ID', 'Date', 'IUCR', 'Primary Type', 'Description', 'Location Description', 'Arrest', 'Domestic', 'Year', 'Latitude', 'Longitude']]    
table_db.head()

,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Year,Latitude,Longitude
0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,2015,41.815117,-87.670000
1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,2015,41.895080,-87.765400
2,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,2015,41.937406,-87.716650
3,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,APARTMENT,False,True,2015,41.881903,-87.755121
4,10224742,09/05/2015 10:55:00 AM,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,2015,41.744379,-87.658431


In [9]:
# Dropping NAN from latitude
# table = table[table['Latitude'].notna()]
# table

In [10]:
# table.reset_index(inplace=True)
# table

In [11]:
table_db['Month'] = table_db['Date'].str[0:2].astype(int)
# table = table.drop(['Date'], axis=1)
table_db.head()

<ipython-input-11-5fb8b2a991d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_db['Month'] = table_db['Date'].str[0:2].astype(int)


,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Year,Latitude,Longitude,Month
0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,2015,41.815117,-87.670000,9
1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,2015,41.895080,-87.765400,9
2,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,2015,41.937406,-87.716650,9
3,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,APARTMENT,False,True,2015,41.881903,-87.755121,9
4,10224742,09/05/2015 10:55:00 AM,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,2015,41.744379,-87.658431,9


# Postgres connection - Chicago crime table

In [12]:
# Remember to change to your Postgres password
rds_connection_string = "postgres:postgres@localhost:5432/Project_03"

# Adding a connection if none are available
engine = create_engine(f'postgresql://{rds_connection_string}', pool_size=10, max_overflow=20)

In [13]:
table_db.to_sql(name='Chicago_crime', con=engine, if_exists='append', index=False)

In [15]:
c = engine.execute('SELECT * FROM "Chicago_crime"').fetchall()
print(c[:10])

[(10224738, '0486', '09/05/2015 01:30:00 PM', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', False, True, 2015, 41.815117282, -87.669999562, 9), (10224739, '0870', '09/04/2015 11:30:00 AM', 'THEFT', 'POCKET-PICKING', 'CTA BUS', False, False, 2015, 41.895080471, -87.765400451, 9), (10224740, '2023', '09/05/2015 12:45:00 PM', 'NARCOTICS', 'POSS: HEROIN(BRN/TAN)', 'SIDEWALK', True, False, 2015, 41.937405765, -87.716649687, 9), (10224741, '0560', '09/05/2015 01:00:00 PM', 'ASSAULT', 'SIMPLE', 'APARTMENT', False, True, 2015, 41.881903443, -87.755121152, 9), (10224742, '0610', '09/05/2015 10:55:00 AM', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', False, False, 2015, 41.744378879, -87.658430635, 9), (10224743, '0620', '09/04/2015 06:00:00 PM', 'BURGLARY', 'UNLAWFUL ENTRY', 'RESIDENCE-GARAGE', False, False, 2015, 41.914635603, -87.681630909, 9), (10224744, '0860', '09/05/2015 01:00:00 PM', 'THEFT', 'RETAIL THEFT', 'GROCERY FOOD STORE', True, False, 2015, 41.851988885, -87.689219118, 9), (102